In [ ]:
import numpy as np
import math
import pyrender
import trimesh
from pygltflib import GLTF2
from transforms3d.quaternions import axangle2quat

# Your existing SkeletonHelper class here
from SkeletonHelperGLTF import SkeletonHelper, compose_matrix, apply_skinning

def load_gltf_mesh_as_trimesh(gltf_path):
    """Loads the glTF mesh using trimesh for visualization."""
    scene = trimesh.load(gltf_path, force='scene')
    return scene

def create_pyrender_scene_from_trimesh(scene):
    pyrender_scene = pyrender.Scene()
    for geom in scene.geometry.values():
        mesh = pyrender.Mesh.from_trimesh(geom)
        pyrender_scene.add(mesh)
    return pyrender_scene

def rotate_joint_360(skeleton: SkeletonHelper, mesh_name, joint_name: str, axis='z', steps=36):
    axis_vecs = {
        'x': np.array([1, 0, 0]),
        'y': np.array([0, 1, 0]),
        'z': np.array([0, 0, 1]),
    }
    axis_vec = axis_vecs.get(axis.lower(), np.array([0, 0, 1]))

    viewer = pyrender.Viewer(create_pyrender_scene_from_trimesh(load_gltf_mesh_as_trimesh(mesh_name)),
                             run_in_thread=True, use_raymond_lighting=True)

    for i in range(steps + 1):
        angle_rad = 2 * np.pi * i / steps
        quat = axangle2quat(axis_vec, angle_rad)
        skeleton.set_joint_rotation_by_name(joint_name, quat)
        skeleton.update()

        # Apply skinning to update mesh
        skinned_mesh = apply_skinning(skeleton, joint_name)

        # Update scene with the new skinned mesh
        viewer.scene.clear()
        mesh = pyrender.Mesh.from_trimesh(skinned_mesh)
        viewer.scene.add(mesh)

        import time
        time.sleep(0.05)  # non-blocking sleep per frame

    viewer.close_external()

# Example usage
gltf_path = "data/raw/human_base_mesh.glb"
gltf = GLTF2().load(gltf_path)
skeleton = SkeletonHelper(gltf)

rotate_joint_360(skeleton, gltf_path, joint_name="DEF-forearm.L", axis='z')

Set joint 163 rotation to [1. 0. 0. 0.]


TypeError: get_accessor_data() missing 1 required positional argument: 'accessor_index'